In [ ]:
%pip install scikeras
%pip install keras-tuner

In [1]:
import pandas as pd
import datetime 
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import keras

import keras_tuner
from sklearn.decomposition import PCA



print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

SEED = 42

tf.keras.utils.set_random_seed(
    SEED
)

# Preprocessing

In [2]:
df_master = pd.read_csv("master_dataset.csv")

df_master

,date,rate,sent_fin_us_uncertainty,sent_fin_us_litigious,sent_fin_us_strong_modal,sent_fin_us_weak_modal,sent_fin_us_constraining,sent_fin_us_optimistic,sent_fin_can_uncertainty,sent_fin_can_litigious,...,index_WGTS.FOPR,index_WGTS.MTLS,index_WGTS.NATURALGAS,index_WGTS.WCC,index_WGTS.WTI,us_bank_rate,sent_bert_us_fin,sent_bert_us_gen,sent_bert_can_fin_sentiment,sent_bert_can_gen_sentiment
0,1955-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
1,1955-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
2,1955-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
3,1955-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
4,1955-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22845,2022-11-01,1.3625,20.0,6.0,6.0,12.0,2.0,-26.0,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,6.25,0.258793,0.719491,NaN,NaN
22846,2022-11-02,1.3632,33.0,6.0,10.0,13.0,14.0,-34.0,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,6.25,0.560365,0.948609,NaN,NaN
22847,2022-11-03,1.3712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,7.00,NaN,NaN,NaN,NaN
22848,2022-11-04,1.3746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,NaN,NaN,NaN,NaN,NaN


In [3]:
df_master['y_exp'] = np.full(df_master.shape[0], -1)
for i, row in df_master.iterrows():
    try:
        two_weeks_from_current_date = datetime.datetime.strptime(row.date, "%Y-%m-%d") + datetime.timedelta(weeks=2)
        futur_rate = list(df_master.loc[df_master['date'] == str(two_weeks_from_current_date.date())].rate)[0]
        df_master.loc[i, 'y_exp' ] = futur_rate
    except: 
        break

df_master = df_master.loc[df_master.y_exp >= 0]
df_master

,date,rate,sent_fin_us_uncertainty,sent_fin_us_litigious,sent_fin_us_strong_modal,sent_fin_us_weak_modal,sent_fin_us_constraining,sent_fin_us_optimistic,sent_fin_can_uncertainty,sent_fin_can_litigious,...,index_WGTS.MTLS,index_WGTS.NATURALGAS,index_WGTS.WCC,index_WGTS.WTI,us_bank_rate,sent_bert_us_fin,sent_bert_us_gen,sent_bert_can_fin_sentiment,sent_bert_can_gen_sentiment,y_exp
7921,1981-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1895
7922,1981-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1863
7923,1981-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1876
7924,1981-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,.,NaN,NaN,NaN,NaN,1.1863
7927,1981-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,2022-10-17,1.3877,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3615
22831,2022-10-18,1.3714,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3625
22832,2022-10-19,1.3741,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3632
22833,2022-10-20,1.3767,20.0,6.0,6.0,12.0,2.0,-26.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3712


In [4]:
df_master.columns

Index(['date', 'rate', 'sent_fin_us_uncertainty', 'sent_fin_us_litigious',
       'sent_fin_us_strong_modal', 'sent_fin_us_weak_modal',
       'sent_fin_us_constraining', 'sent_fin_us_optimistic',
       'sent_fin_can_uncertainty', 'sent_fin_can_litigious',
       'sent_fin_can_strong_modal', 'sent_fin_can_weak_modal',
       'sent_fin_can_constraining', 'sent_fin_can_optimistic',
       'ti_rate_rsi_14', 'ti_rate_stochrsi_14', 'ti_rate_macd_12_26',
       'ti_rate_adx_14', 'ti_rate_williams_%R', 'ti_rate_cci', 'ti_rate_atr',
       'ti_rate_utlimate_oscillator', 'ti_rate_roc', 'interest_rate_can',
       'index_W.BCPI', 'index_W.BCNE', 'index_W.ENER', 'index_W.MTLS',
       'index_W.FOPR', 'index_W.AGRI', 'index_W.FISH', 'index_WGTS.AGRI',
       'index_WGTS.BRENT', 'index_WGTS.COAL', 'index_WGTS.FISH',
       'index_WGTS.FOPR', 'index_WGTS.MTLS', 'index_WGTS.NATURALGAS',
       'index_WGTS.WCC', 'index_WGTS.WTI', ' us_bank_rate', 'sent_bert_us_fin',
       'sent_bert_us_gen', 'sent_b

In [5]:
df_master= df_master.dropna()
df_master= df_master.reset_index(drop=True)
df_master = df_master.loc[df_master[' us_bank_rate'] != '.']

In [6]:
train = df_master.loc[df_master.date < "2022-08-01"]
test = df_master.loc[df_master.date >= "2022-08-01"]

display( train.shape, test.shape)

(6843, 46)

(59, 46)

In [7]:
train.date = pd.to_datetime(train.date).astype('int64')/ 10**9
test.date = pd.to_datetime(test.date).astype('int64')/ 10**9


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


# Baseline - Mean

In [72]:
x_t, x_v, y_t, y_v = train_test_split(X_train, y_train, test_size=0.15, shuffle=True, random_state=SEED)


y_pred = np.full(test_y.shape[0] , x_t.rate.mean())
mse = mean_squared_error(test_y.y_exp.to_list(), y_pred)
print(f" Mean squared error: {mse}")

 Mean squared error: 0.0059913897484831485


# DNN Model

In [9]:
sorted_index = ['interest_rate_can',
 'index_WGTS.BRENT',
 'rate',
 'ti_rate_rsi_14',
 'sent_fin_can_litigious',
 'ti_rate_stochrsi_14',
 'sent_fin_us_strong_modal',
 'ti_rate_williams_%R',
 'index_WGTS.COAL',
 'ti_rate_macd_12_26',
 'sent_fin_us_uncertainty',
 'sent_fin_can_uncertainty',
 'index_W.FISH',
 'ti_rate_atr',
 'sent_fin_us_weak_modal',
 'sent_bert_can_fin_sentiment',
 'index_WGTS.WCC',
 'index_W.ENER',
 'sent_bert_us_gen',
 'sent_bert_can_gen_sentiment',
 'sent_fin_us_litigious',
 'sent_bert_us_fin',
 'index_W.FOPR',
 'ti_rate_utlimate_oscillator',
 'index_WGTS.FOPR',
 'sent_fin_can_strong_modal',
 'ti_rate_adx_14',
 'ti_rate_roc',
 'ti_rate_cci',
 'sent_fin_can_weak_modal',
 'sent_fin_can_optimistic',
 'sent_fin_us_optimistic',
 'index_WGTS.NATURALGAS',
 'sent_fin_can_constraining',
 'us_bank_rate',
 'index_WGTS.MTLS',
 'index_WGTS.AGRI',
 'index_W.MTLS',
 'index_W.BCNE',
 'index_WGTS.WTI',
 'index_WGTS.FISH',
 'sent_fin_us_constraining',
 'index_W.AGRI',
 'index_W.BCPI']

In [39]:
X_train = train[sorted_index[:5]].astype(float)
y_train = train.loc[:, train.columns == "y_exp"]

## Hyper-parameter tunning


In [ ]:
def build_tune_model(hp):
	# create model
	input_dim = len(X_train.columns)
	model = Sequential()
	model.add(Dense( hp.Int("layer1", min_value=1, max_value=20, step=1), input_shape=(input_dim,), kernel_initializer='normal', activation='relu'))
    
	nb_layer = hp.Int("num_layers", 0, 4)
	if nb_layer != 0:
		for i in range(0, nb_layer):
			model.add(
                layers.Dense(
                    units=hp.Int(f"units_{i}", min_value=1, max_value=20, step=1),
                    activation='relu',
                    kernel_initializer='normal'
                )
            )
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	# le = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
	# le = hp.Choice('lr', [1.0, 1e-2, 1e-4, 1e-3])
	model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=0.001))
    
	return model

In [45]:
tuner = keras_tuner.RandomSearch(
    build_tune_model,
    objective='val_loss',
    overwrite=True,
    max_trials=10)

x_t, x_v, y_t, y_v = train_test_split(X_train, y_train, test_size=0.15, shuffle=True, random_state=SEED)


monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, 
                        verbose=1, mode='auto', restore_best_weights=True)

tuner.search(x_t, y_t, batch_size=5, epochs=100, validation_data=(x_v, y_v), callbacks=[monitor])
best_model = tuner.get_best_models()[0]
best_model.summary()

Trial 10 Complete [00h 01m 08s]
val_loss: 0.0003767136367969215

Best val_loss So Far: 0.00037268581218086183
Total elapsed time: 00h 10m 39s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 35        
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


## Best model training

In [77]:
# define base model
def baseline_model(input_dim):
	# create model
	model = Sequential()
	model.add(Dense(5, input_shape=(input_dim,), kernel_initializer='normal', activation='relu'))
	model.add(Dense(5,  kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=0.001))
	# model.compile(loss='mean_squared_error', optimizer='adam')
    
	return model

In [89]:
model = baseline_model(X_train.shape[1])

x_t, x_v, y_t, y_v = train_test_split(X_train, y_train, test_size=0.15, shuffle=True, random_state=SEED)



monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, 
                        verbose=1, mode='auto', restore_best_weights=True)

model.fit(x=x_t, y=y_t, batch_size=10, epochs=100, validation_data=(x_v, y_v), callbacks=[monitor])


Epoch 1/100
582/582 [==============================] - 2s 3ms/step - loss: 0.3066 - val_loss: 0.0547
Epoch 2/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0519 - val_loss: 0.0415
Epoch 3/100
582/582 [==============================] - 2s 4ms/step - loss: 0.0393 - val_loss: 0.0309
Epoch 4/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0291 - val_loss: 0.0227
Epoch 5/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0209 - val_loss: 0.0161
Epoch 6/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0143 - val_loss: 0.0111
Epoch 7/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0097 - val_loss: 0.0080
Epoch 8/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0074 - val_loss: 0.0066
Epoch 9/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0063 - val_loss: 0.0060
Epoch 10/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0057 - val_lo

## Evaluation

In [90]:
y_pred = [p[0] for p in model.predict(test[X_train.columns].astype(float))]

test_y = test.loc[:, train.columns == "y_exp"]

mse = mean_squared_error(test_y.y_exp.to_list(), y_pred)
print(f" Mean squared error: {mse}")


2/2 [==============================] - 0s 3ms/step
 Mean squared error: 0.0003749132069074687


In [91]:
import shutil
model.save("Model_DNN_5-5-1")
shutil.make_archive("Model_DNN_5-5-1", 'zip', "Model_DNN_1")

'/content/Model_DNN_5-5-1.zip'